## In this notebook, you will learn how to create model serving endpoints that deploy and serve foundation models.

Mosaic AI Model Serving supports the following models:

- External models. These are foundation models that are hosted outside of Databricks. Endpoints that serve external models can be centrally governed and customers can establish rate limits and access control for them. Examples include foundation models like OpenAI's GPT family and Anthropic's Claude.

- State-of-the-art open foundation models made available by Foundation Model APIs. These models are curated foundation model architectures that support optimized inference. Base models, like Meta-Llama-3.1-70B-Instruct, GTE-Large, and Mistral-7B are available for immediate use with pay-per-token pricing. Production workloads, using base or fine-tuned models, can be deployed with performance guarantees using provisioned throughput.

Model Serving provides the following options for model serving endpoint creation:

- The Serving UI
- REST API
- MLflow Deployments SDK

For creating endpoints that serve traditional ML or Python models, see Create custom model serving endpoints.

## Create a foundation model serving endpoint

You can create an endpoint that serves fine-tuned variants of foundation models made available using Foundation Model APIs provisioned throughput. See Create your provisioned throughput endpoint using the REST API.

For foundation models that are made available using Foundation Model APIs pay-per-token, Databricks automatically provides specific endpoints to access the supported models in your Databricks workspace. To access them, select the Serving tab in the left sidebar of the workspace. The Foundation Model APIs are located at the top of the Endpoints list view.

For querying these endpoints, see Use foundation models.


## 1 configuration of the model endpoints

In this demo we will define 2 models, 
- The chat model gpt_4o_mini, quite small , will be enough for the demos.
- The embedding model, dedicated to the retriever component. It will be used for the meaning proximity between sentences or documents.


In [0]:
models_config = [
    {               # Here is the first LLM, it's an openai model 
        "name":"chat_gpt_4o_mini", # Name of the endpoint that will be created
        "config": {
            "served_entities": [
                {
                    "name": "chat",
                    "external_model": { # openai is a commercial provider, so you will use the api for each submitted request
                        "name": "gpt-4o-mini", # name of the model selected, here, the size matters so mini  is a good choice for a demo
                        "provider": "openai", # provider; the provider is the incontournable openai
                        "task": "llm/v1/chat",  # Category of the task, here, it's a llm, version 1 dedicated to chat.
                        "openai_config": {
                            "openai_api_key": "{{secrets/llm_secrets/openai_api_key}}", # You must have already registered the key in the llm_secret store
                        },
                    },
                }
            ],
        },
    },
    {               # embeddings text-embedding-3-small
        "name":"text_embedding_3_large",
        "config": {
            "served_entities": [
                {
                    "name": "embeddings",
                    "external_model": {
                        "name": "text-embedding-3-large",
                        "provider": "openai",
                        "task": "llm/v1/embeddings", # Category of the task, here, it's a llm, version 1 dedicated to embedding.
                        "openai_config": {
                            "openai_api_key": "{{secrets/llm_secrets/openai_api_key}}",
                        },
                    },
                }
            ],
        },
    },
]

## 2 Deploy the endpoints

After deployed the agent will be available through endpoints.

The creation can take a few minutes, the state will give information upon the readyness of the agent

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
for config in models_config :
    try:
        
        endpoint = client.create_endpoint(**config)
    
        print("Endpoint created !")
        print(f"Name: {endpoint.get('name')}")
        print(f"ID: {endpoint.get('id')}")
        print(f"State: {endpoint.get('state')}")

    except Exception as e:
        print(f"Error while creating the endpoint: {str(e)}")

## 3 Serving
On the serving menu, you have access to your model endpoint library.


This last cell can be used if don't need a model anymore.

In [0]:
#client.delete_endpoint("chat_gpt_4o_mini")